In this Jupyter notebook, we will do the final holdout to both M5 challenges, and we will study the several ARIMA model that we did. Mainly, we will see what variables did we use and how many model with just zeroes we have. 

In [1]:
import time
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

In [25]:
path_holdout = 'C:\\Users\\maxwi\\Python\\Kaggle\\M5 Forecasting - Accuracy\\Modelo 1\\model\\result\\holdout_by_id\\'
path_summary = 'C:\\Users\\maxwi\\Python\\Kaggle\\M5 Forecasting - Accuracy\\Modelo 1\\model\\result\\summary_by_id\\'

sales_train_validation = pd.read_csv('sales_train_validation.csv') #Only used to take the products ids.
products_ids = sales_train_validation['id'].unique()
products_ids_size = len(products_ids)
del sales_train_validation

In [26]:
#Make the final summary and holdout to M5 Forecast Accuracy 

stoped_at = products_ids[0]  #  Used to continue the code from the last product saved. For safety, we will run the last product saved again.

holdout_accuracy = pd.read_hdf(path_holdout + stoped_at + '.h5')
summary = pd.read_hdf(path_summary + stoped_at + '.h5')

progress = 0  #Usefull to see the progress of the code. 
progress_1000 = 1
start = time.time()

for product in products_ids[1:]:
    holdout_accuracy_aux = pd.read_hdf(path_holdout + product + '.h5')
    summary_aux = pd.read_hdf(path_summary + product + '.h5')
    
    holdout_accuracy = pd.concat([holdout_accuracy, holdout_accuracy_aux], ignore_index=True)
    summary = pd.concat([summary, summary_aux], ignore_index=True)

    progress += 1
    if progress == progress_1000 * 1000:

        progress_per = round(progress / products_ids_size, 4)
        print(progress_per)
        progress_1000 +=1
        
        end = time.time()
        elapsed = int(round(end - start, 0))
        total_run_time =  int(round(elapsed / (progress_per), 0))
        time_to_finish = int(round(elapsed / (progress_per), 0)) - elapsed
        print('Elapsed: {:02d}:{:02d}:{:02d}'.format(elapsed // 3600, (elapsed % 3600 // 60), elapsed % 60))
        print('Total run time: {:02d}:{:02d}:{:02d}'.format(total_run_time // 3600, (total_run_time % 3600 // 60), total_run_time % 60))
        print('Time to finish: {:02d}:{:02d}:{:02d}'.format(time_to_finish // 3600, (time_to_finish % 3600 // 60), time_to_finish % 60))
        print()

holdout_accuracy.to_hdf('holdout_accuracy.h5', key = product, mode = 'w')
summary.to_hdf('summary.h5', key = product, mode = 'w')

0.0328
Elapsed: 00:00:22
Total run time: 00:11:11
Time to finish: 00:10:49

0.0656
Elapsed: 00:00:49
Total run time: 00:12:27
Time to finish: 00:11:38

0.0984
Elapsed: 00:01:20
Total run time: 00:13:33
Time to finish: 00:12:13

0.1312
Elapsed: 00:01:57
Total run time: 00:14:52
Time to finish: 00:12:55

0.164
Elapsed: 00:02:37
Total run time: 00:15:57
Time to finish: 00:13:20

0.1968
Elapsed: 00:03:20
Total run time: 00:16:56
Time to finish: 00:13:36

0.2296
Elapsed: 00:04:08
Total run time: 00:18:00
Time to finish: 00:13:52

0.2624
Elapsed: 00:04:58
Total run time: 00:18:56
Time to finish: 00:13:58

0.2952
Elapsed: 00:05:53
Total run time: 00:19:56
Time to finish: 00:14:03

0.328
Elapsed: 00:06:52
Total run time: 00:20:56
Time to finish: 00:14:04

0.3608
Elapsed: 00:07:54
Total run time: 00:21:54
Time to finish: 00:14:00

0.3936
Elapsed: 00:08:59
Total run time: 00:22:49
Time to finish: 00:13:50

0.4264
Elapsed: 00:10:08
Total run time: 00:23:46
Time to finish: 00:13:38

0.4592
Elapsed

In [34]:
holdout_accuracy.columns = ['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14'
                           , 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']
holdout_accuracy.to_csv("holdout_accuracy.csv", index=False)

Using one ARIMA per product, we obtained a score of 3.92318.
Probably, the biggest problem of our model was the automatization of the parameters 'p' an 'q' in the ARIMA model.
Not only did we not use any otimization algorithm to set these parameters, but we also received a lot of warnings saying that our 'p' and 'q' were not optimal (we can see that by the variable error_fit in our summary).

In [85]:
variables_used = summary.drop('product', axis = 1).sum()
print(round(variables_used['error_fit'] / len(products_ids), 4))

0.1674


In 16.74% of our ARIMA models, we had a problem when calculating the betas and were forced to set some initial parameters.
However, this indicate that we could not minimize the error of our betas.

One easy way to fix this problem would be to use some algorithm to automatically choose the parameters of the ARIMA model. Nevertheless, this is pratically impossible to do with only a personal computer because it would be time consuming.

In [88]:
variables_used.sort_values()

q_parameter                        0.000000e+00
i_parameter                        0.000000e+00
p_parameter                        0.000000e+00
price_change_max_t52_bin_0.25++    2.200000e+01
price_change_max_t26_bin_0.25++    2.300000e+01
                                       ...     
sum_forecast                       7.252540e+05
number_of_observation_forecast     1.707440e+06
number_of_observation_train        4.602796e+07
sum_train                          6.569541e+07
AIC                                1.337783e+08
Length: 250, dtype: float64

In [105]:
#Most used variables
variables_used[(variables_used > 2000) &(variables_used < 11000)].sort_values()


season_4                    2058.000000
month_11                    2074.000000
month_10                    2125.000000
month_9                     2127.000000
demand_rolling_std_t90      2133.000000
wday_7                      2196.000000
demand_rolling_skew_t30     2213.000000
month_fase_1                2225.000000
season_1                    2237.000000
demand_rolling_std_t7       2256.000000
season_2                    2261.000000
wday_2                      2836.000000
wday_1                      2879.000000
wday_3                      3078.000000
snap                        3381.000000
month_fase_3                3435.000000
wday_6                      3454.000000
week_fase_3                 3543.000000
demand_rolling_std_t365     3580.000000
wday_4                      4124.000000
demand_rolling_std_t30      4196.000000
error_fit                   5104.000000
std_test                    8572.603732
week_fase_1                10722.000000
dtype: float64

In [98]:
#least used variables.
variables_used[variables_used < 300]

Sporting                                270.0
Sporting_near                           294.0
price_change_t1_bin_--0.25              218.0
price_change_t1_bin_-0.25-0.10          190.0
price_change_t1_bin_-0.10-0.5           139.0
price_change_t1_bin_-0.05-0             234.0
price_change_t1_bin_0.05-0.10           124.0
price_change_t1_bin_0.10-0.25           140.0
price_change_t1_bin_0.25++               32.0
price_change_max_t4_bin_--0.25          143.0
price_change_max_t4_bin_-0.25-0.10      118.0
price_change_max_t4_bin_-0.10-0.5        80.0
price_change_max_t4_bin_-0.05-0         131.0
price_change_max_t4_bin_0.05-0.10       150.0
price_change_max_t4_bin_0.10-0.25       170.0
price_change_max_t4_bin_0.25++          117.0
price_change_max_t13_bin_--0.25         114.0
price_change_max_t13_bin_-0.25-0.10     114.0
price_change_max_t13_bin_-0.10-0.5       70.0
price_change_max_t13_bin_-0.05-0         82.0
price_change_max_t13_bin_0.05-0.10      146.0
price_change_max_t13_bin_0.10-0.25

We did not save the used 'p', 'i' and 'q' parameter in our summary.
We can see that we used all the variables that we construct. However, some variables, especially the ones related to price change, were used in less than one percent of the models.

On the other hand, the most used variables were variables related to time and the mean of the lagged demand. 
Here we have another problem. Both kind of variables already are considered in the ARIMA model. 
In general, the mean of the lagged demand does not necessary imply in a problem because we avoided using a 'q' parameter greater than 20. Nonetheless, several variables related to time were used, this could had created a correlation problem in our models. In the construction of our models, we guarantee that none of the exoneous variables were correlated among them, but we did not verify if we had some problem between the exogenous variables and the 'p' paramer of our ARIMA model. 